# Some exploratory data anaylisis, to check that the normalisation is working

In [18]:
from configs import avail_models, get_avail_splits
from models import norm_vals
from video_dataset import get_wlasl_info

## Setup

### Model

Available models:

In [5]:
av_models = avail_models()
for m in av_models:
    print(m)

S3D
R3D_18
R(2+1)D_18
Swin3D_T
Swin3D_S
Swin3D_B
MViTv2_S
MViTv1_B


Chosen model: 

In [11]:
model_name = av_models[0]
print(model_name)

S3D


Norm values: 

In [13]:
model_info = norm_vals(model_name)
print(model_info)

{'mean': (0.43216, 0.394666, 0.37645), 'std': (0.22803, 0.22145, 0.216989)}


### Dataset

Available splits

In [ ]:
av_splits = get_avail_splits()
for s in av_splits:
    print(s)

asl100
asl2000
asl300
asl1000


Chosen split, and set

In [16]:
split_name = av_splits[0]
print(split_name)

asl100


Dataset info

In [19]:
train_info = get_wlasl_info(split_name, set_name="train")
print(train_info)


{'root': PosixPath('../data/WLASL/WLASL2000'), 'labels': PosixPath('preprocessed/labels/asl100'), 'label_suff': 'fixed_frange_bboxes_len.json', 'set_name': 'train'}


Dataloader